In [126]:
import arcpy
import csv
import pandas as pd

Environments

In [127]:
arcpy.env.workspace = 'C:/MobilityRecord'
csvfile = arcpy.env.workspace + "/domains.csv"
df_csv = pd.read_csv(csvfile,sep=';', encoding = 'latin1')
df_csv

,FIELD,ALIAS,LENGHT,DOMNAME,DOMDESCRIPTION,VALUECODE,VALUEDESCRIPTION
0,STADTTEIL,District,50,Hubs_District,District,CHA,Charlottenburg-Wilmersdorf
1,STADTTEIL,District,50,Hubs_District,District,FRI,Friedrichshain-Kreuzberg
2,STADTTEIL,District,50,Hubs_District,District,LIC,Lichtenberg
3,STADTTEIL,District,50,Hubs_District,District,MAR,Marzahn-Hellersdorf
4,STADTTEIL,District,50,Hubs_District,District,MIT,Mitte
5,STADTTEIL,District,50,Hubs_District,District,NEU,Neukölln
6,STADTTEIL,District,50,Hubs_District,District,PANK,Pankow
7,STADTTEIL,District,50,Hubs_District,District,REI,Reinickendorf
8,STADTTEIL,District,50,Hubs_District,District,SPA,Spandau
9,STADTTEIL,District,50,Hubs_District,District,STE,Steglitz-Zehlendorf


In [128]:
url = 'https://de.wikipedia.org/wiki/Verwaltungsgliederung_Berlins'
df_wiki = pd.read_html(url)
districts = df_wiki[0]

districts['FIELD'] = 'STADTTEIL'
districts['ALIAS'] = 'District'
districts['LENGHT'] = 50
districts['DOMNAME'] = 'Hubs_District'
districts['DOMDESCRIPTION'] = 'District'

districts['VALUECODE'] = districts['Bezirk'].str[:3].str.upper()
districts['VALUEDESCRIPTION'] = districts['Bezirk']


data =[districts['FIELD'], districts['ALIAS'], districts['LENGHT'], districts['DOMNAME'],districts['DOMDESCRIPTION'],
      districts['VALUECODE'],districts['VALUEDESCRIPTION']]
columns = ['FIELD', 'ALIAS', 'LENGHT', 'DOMNAME', 'DOMDESCRIPTION','VALUECODE','VALUEDESCRIPTION']

df_districts = pd.concat(data, axis=1, keys = columns)
df_districts 

,FIELD,ALIAS,LENGHT,DOMNAME,DOMDESCRIPTION,VALUECODE,VALUEDESCRIPTION
0,STADTTEIL,District,50,Hubs_District,District,MIT,Mitte
1,STADTTEIL,District,50,Hubs_District,District,FRI,Friedrichshain-Kreuzberg
2,STADTTEIL,District,50,Hubs_District,District,PAN,Pankow
3,STADTTEIL,District,50,Hubs_District,District,CHA,Charlottenburg-Wilmersdorf
4,STADTTEIL,District,50,Hubs_District,District,SPA,Spandau
5,STADTTEIL,District,50,Hubs_District,District,STE,Steglitz-Zehlendorf
6,STADTTEIL,District,50,Hubs_District,District,TEM,Tempelhof-Schöneberg
7,STADTTEIL,District,50,Hubs_District,District,NEU,Neukölln
8,STADTTEIL,District,50,Hubs_District,District,TRE,Treptow-Köpenick
9,STADTTEIL,District,50,Hubs_District,District,MAR,Marzahn-Hellersdorf


In [129]:
df = df_csv.append(df_districts)
df

,FIELD,ALIAS,LENGHT,DOMNAME,DOMDESCRIPTION,VALUECODE,VALUEDESCRIPTION
0,STADTTEIL,District,50,Hubs_District,District,CHA,Charlottenburg-Wilmersdorf
1,STADTTEIL,District,50,Hubs_District,District,FRI,Friedrichshain-Kreuzberg
2,STADTTEIL,District,50,Hubs_District,District,LIC,Lichtenberg
3,STADTTEIL,District,50,Hubs_District,District,MAR,Marzahn-Hellersdorf
4,STADTTEIL,District,50,Hubs_District,District,MIT,Mitte
5,STADTTEIL,District,50,Hubs_District,District,NEU,Neukölln
6,STADTTEIL,District,50,Hubs_District,District,PANK,Pankow
7,STADTTEIL,District,50,Hubs_District,District,REI,Reinickendorf
8,STADTTEIL,District,50,Hubs_District,District,SPA,Spandau
9,STADTTEIL,District,50,Hubs_District,District,STE,Steglitz-Zehlendorf


In [130]:
gdb = arcpy.env.workspace + '/MobilityRecord.gdb'
inFeatures = gdb +'/MobilityHubs'

Add Fields

In [131]:
df_fields = df.drop_duplicates(subset=["FIELD"],keep ='first')

for Field,Leng in zip(df_fields['FIELD'], df_fields['LENGHT']):
    arcpy.AddField_management(inFeatures, Field, "TEXT", field_length = Leng)

Create Domains

In [132]:
Domains = df['DOMNAME'].unique()
df_domains = df.drop_duplicates(subset=["DOMNAME"],keep ='first').drop(columns=(['ALIAS','LENGHT','VALUECODE','VALUEDESCRIPTION']),axis=0)

In [133]:
df_domains

,FIELD,DOMNAME,DOMDESCRIPTION
0,STADTTEIL,Hubs_District,District
12,SHARED_MOBILITY,Hubs_SharedMob,Shared Mobility
17,OPNV,Hubs_OPNV,Public Transport


In [134]:
for row in df_domains.index:
    print(df_domains['DOMNAME'][row])
    arcpy.CreateDomain_management(gdb, df_domains['DOMNAME'][row], df_domains['DOMDESCRIPTION'][row], "TEXT", "CODED")

Hubs_District
Hubs_SharedMob
Hubs_OPNV


Add values to domains

In [135]:
df_domValues = df.drop(columns=(['FIELD','ALIAS','LENGHT','DOMDESCRIPTION']),axis=0).reset_index()
df_domValues
for row in df_domValues.index:
    arcpy.management.AddCodedValueToDomain(gdb, df_domValues['DOMNAME'][row], df_domValues['VALUECODE'][row], df_domValues['VALUEDESCRIPTION'][row])

In [136]:
for row in df_domains.index:
    arcpy.AssignDomainToField_management(inFeatures, df_domains['FIELD'][row], df_domains['DOMNAME'][row])